In [ ]:
# pip install ipykernel ipywidgets
# uv pip install datasets langchain langchain-openai python-dotenv

# Dataset Loading

- [Naver 경제, IT 뉴스기사 요약 데이터셋](https://huggingface.co/datasets/daekeun-ml/naver-news-summarization-ko)
  - NLP 모델 학습을 위해 Naver 뉴스 기사를 크롤링한 데이터셋
  - **2022년 7월 1일 ~ 7월 10일** 기간 동안의 **IT와 경제 기사**를 수집
  - **Train, Test, Validation** Dataset으로 구성됨

In [1]:
from datasets import load_dataset

# Hugging Face 데이터셋 로드
dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")
dataset


README.md:   0%|          | 0.00/787 [00:00<?, ?B/s]

c:\Documents\SKN21_inst\11_llm_파인튜닝\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\datasets--daekeun-ml--naver-news-summarization-ko. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train.csv:   0%|          | 0.00/66.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/22194 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2466 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2740 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [3]:
trainset = dataset['train']
trainset

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
    num_rows: 22194
})

In [5]:
df_train = trainset.to_pandas()
df_train.shape

(22194, 7)

In [6]:
df_train.head()

,date,category,press,title,document,link,summary
0,2022-07-03 17:14:37,economy,YTN,추경호 중기 수출지원 총력 무역금융 40조 확대,앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 ...,https://n.news.naver.com/mnews/article/052/000...,"올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록한 가운데, ..."
1,2022-07-04 08:07:12,economy,아시아경제,해산물·주류 무제한 인터컨티넨탈 뷔페 여름 한정 페스타 연다,문어 랍스터 대게 갑오징어 새우 소라 등 해산물 활용 미국식 해물찜 시푸드 보일 준...,https://n.news.naver.com/mnews/article/277/000...,인터엑스 1층 뷔페 레스토랑 브래서리는 오는 6일부터 8월31일까지 쿨 섬머 페스타...
2,2022-07-05 18:01:01,IT과학,동아사이언스,한국인 생활 변화시킨 의사 마지막까지 영향력 컸던 과학자 이호왕 고려대 명예교수 소천,한탄바이러스 발견 노벨상 유력 후보로 자주 거론 한국을 대표하는 의학자이자 미생물학...,https://n.news.naver.com/mnews/article/584/000...,이 이호왕 고려대 명예교수는 바이러스의 병원체와 진단법 백신까지 모두 개발한 한국을...
3,2022-07-01 08:51:12,economy,뉴시스,에디슨이노 이승훈 제이스페이스 대표 사내이사 선임,기사내용 요약 우주발사체 사업 본격화 서울 뉴시스 김경택 기자 에디슨이노가 우주발사...,https://n.news.naver.com/mnews/article/003/001...,에디슨이노는 1일 임시주주총회를 통해 사명을 이노시스 로 변경하고 이승훈 제이스페이...
4,2022-07-01 16:11:01,economy,머니투데이,SK바사 해외 사업 조직 개편… 글로벌 탑티어 기업으로 성장 박차,SK바이오사이언스가 글로벌 사업의 고도화를 위해 조직 개편을 단행했다. SK바이오사...,https://n.news.naver.com/mnews/article/008/000...,SK바이오사이언스가 글로벌 사업의 고도화를 위해 기존 해외사업개발실을 백신사업뿐만 ...


In [7]:
df_train['category'].value_counts()

category
economy    17088
IT과학        5106
Name: count, dtype: int64

In [8]:
# 경제 기사만 조회 + 행 index 정리.
df = df_train.query("category=='economy'").reset_index(drop=True)
df.shape

(17088, 7)

In [9]:
df.head()

,date,category,press,title,document,link,summary
0,2022-07-03 17:14:37,economy,YTN,추경호 중기 수출지원 총력 무역금융 40조 확대,앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 ...,https://n.news.naver.com/mnews/article/052/000...,"올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록한 가운데, ..."
1,2022-07-04 08:07:12,economy,아시아경제,해산물·주류 무제한 인터컨티넨탈 뷔페 여름 한정 페스타 연다,문어 랍스터 대게 갑오징어 새우 소라 등 해산물 활용 미국식 해물찜 시푸드 보일 준...,https://n.news.naver.com/mnews/article/277/000...,인터엑스 1층 뷔페 레스토랑 브래서리는 오는 6일부터 8월31일까지 쿨 섬머 페스타...
2,2022-07-01 08:51:12,economy,뉴시스,에디슨이노 이승훈 제이스페이스 대표 사내이사 선임,기사내용 요약 우주발사체 사업 본격화 서울 뉴시스 김경택 기자 에디슨이노가 우주발사...,https://n.news.naver.com/mnews/article/003/001...,에디슨이노는 1일 임시주주총회를 통해 사명을 이노시스 로 변경하고 이승훈 제이스페이...
3,2022-07-01 16:11:01,economy,머니투데이,SK바사 해외 사업 조직 개편… 글로벌 탑티어 기업으로 성장 박차,SK바이오사이언스가 글로벌 사업의 고도화를 위해 조직 개편을 단행했다. SK바이오사...,https://n.news.naver.com/mnews/article/008/000...,SK바이오사이언스가 글로벌 사업의 고도화를 위해 기존 해외사업개발실을 백신사업뿐만 ...
4,2022-07-01 21:48:04,economy,경향신문,금융당국 “증시 변동성 완화 조치”,4일부터 석달간 증권사 신용융자담보비율 유지의무 면제 금융당국이 코스피지수가 장중 ...,https://n.news.naver.com/mnews/article/032/000...,1일 1일 금융위원회는 증권 유관기관과 금융시장합동점검회의를 열고 코스피지수가 장중...


# 데이터셋 만들기
- 뉴스기사 제목과 뉴스기사를 이용해 그 기사에 영향을 받는 주가종목을 추론하는 모델을 만든다.
- 데이터 구성
  - **입력**: 뉴스 기사 제목, 뉴스 기사
  - **출력**
    1. 뉴스기사가 주식시장에 영향을 주는지 여부
    1. 이 기사에 부정적인 영향을 받는 회사들과 이유
    1. 이 기사에 긍정적인 영향을 받는 회사들과 이유
    1. 뉴스기사 요약
- Dataset 생성
  - Label(정답)을 **LLM을 이용해 생성**한다.
  - **LLM을 이용해 데이터셋을 만든 이후 그 결과를 눈으로 검토해야 한다.**

## Dataset 생성 Chain 구성

In [10]:
from tqdm import tqdm
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser

from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
# 프롬프트 템플릿

template = '''# Instruction
당신은 금융 뉴스의 핵심 내용을 요약해 설명하고, 뉴스가 특정 상장 종목에 미치는 긍정/부정 영향 여부, 이유, 근거 등을 분석하는 금융 분석 전문가입니다.
사용자에 의해 입력된 뉴스 기사를 분석해서 **한국에 상장된 주식 종목에 영향을 주는지 판단**하고, Output Indicator에 제시된 기준에 따라 구조화된 JSON 형식으로 결과를 출력하세요.

## 분석 기준
1. 뉴스가 **한국 주식 종목에 영향을 주는지 판단**하세요.
2. 영향을 준다면 다음 항목을 출력하세요.
   - `"is_stock_related": true`
   - 뉴스에 **긍정적** 영향을 받는 **회사이름들**
   - 뉴스에 **부정적** 영향을 받는 **회사이름들**
   - 뉴스가 각 회사에 **긍정적 또는 부정적 영향을 주는지 이유**
     - 반드시 **뉴스기사에 언급된 내용 기반으로 작성한다.** 뉴스기사에 없는 내용을 꾸며서 임의로로 작성하지 않습니다.
     - `None`, 유추, 추정, 일반 논평 금지합니다.
   - **뉴스 내용 요약** (3줄 이내)
3. 뉴스가 한국 주식 종목에 영향을 주지 않는다면 다음 항목을 출력하세요.
   - `"is_stock_related": false`
   - **뉴스 내용 요약** (3줄 이내)

# 입력 데이터(뉴스기사)

{input}


# 출력 지시사항 (Output Indicator)

- {format_instructions}

## 출력 조건:
- 뉴스에 영향을 받은 회사들은 **반드시 한국 증시에 상장된 종목** 이어야 합니다.
- 뉴스에 있는 내용만 출력결과에 포함시킵니다.
- 긍정/부정 종목은 실제 뉴스기사에 영향을 받는 회사들만 포함하세요.
- 모든 문자열은 큰따옴표(`"`)로 감쌉니다.
- 문자열 안에 따옴표가 필요하면 작은따옴표(`'`)를 사용합니다.
- 모든 키(Key)는 출력 지시사항에 명시된 property들과 정확히 일치해야 합니다.
- `"positive_reasons"` 및 `"negative_reasons"`의 값은 `None`이 될 수 없습니다.
- json format을 잘 지켜 응답데이터를 만듭니다. 배열이나 object의 마지막 항목 뒤에 `,` 를 붙이지 마세요.
- 오직 유효한 JSON 문자열(UTF-8, RFC8259 준수)만 출력합니다.
- 절대 다른 텍스트, 주석, 설명, 코드 블록 표기(```) 들을 추가하면 안 됩니다.

## 출력 예시 (Examples)

### 뉴스가 특정 주식종목들에 **긍정적 영향이 주는 경우**:
{{'is_stock_related': true,
 'negative_reasons': [],
 'negative_stocks': [],
 'positive_reasons': [{{'세라젬': '루게릭병 환우 지원 캠페인 후원과 의료가전 지원 등 사회공헌활동을 통해 기업 이미지와 브랜드 가치가 긍정적으로 부각됨'}}],
 'positive_stocks': ['세라젬'],
 'summary': '세라젬이 루게릭병 환우를 위한 아이스버킷 챌린지 런 행사를 후원하며 의료가전과 건강기능식품 등을 지원했다. 캠페인은 루게릭병 환우 지원과 기부 문화 확산을 목표로 한다. 세라젬은 다양한 사회공헌활동을 지속하고 있다.'
}}

### 뉴스의 내용이 특정 주식종목들에 **부정적 영향이 주는 경우**:
{{
    "is_stock_related": true,
    "positive_stocks": [],
    "positive_reasons": [],
    "negative_stocks": [
        "포스코",
        "현대제철"
    ],
    "negative_reasons": [
        {{"포스코": "정부가 수입규제국 조사에 적극 대응하고 비관세장벽 해소를 위해 민관 협력 강화 방침을 밝혀 철강 분야에서 수출 피해 최소화 기대"}},
        {{"현대제철": "철강·금속 품목에 대한 수입규제 대응 강화로 불합리한 무역제한 조치 개선 가능성이 높아져 수출 환경 개선 기대"}}
    ],
    "summary": "산업부는 수입 규제국의 조사에 대응하고 비관세장벽 해소를 위한 협의를 진행했다. 규제 대상 국가는 26개국, 건수는 199건에 달한다."
}}

### **뉴스기사가 주식 종목과 관련 없는 경우**:
{{
    "is_stock_related": false,
    "positive_stocks": [],
    "positive_reasons": [],
    "negative_stocks": [],
    "negative_reasons": [],
    "summary": "정황근 농림축산식품부 장관이 단순가공식품 부가가치세 면제 시행 상황을 점검했다. 된장, 고추장 코너를 방문하며 현장을 살폈다."
}}'''

In [25]:
# Output Parser 정의
class SummarySchema(BaseModel):
    is_stock_related: bool = Field(description="한국 주식 종목과 관련있는 뉴스인지 여부")
    positive_stocks: list[str] = Field(description="뉴스기사에 긍정적인 영향을 받는 회사들의 이름들.")
    positive_reason: list[dict[str,str]] = Field(description='뉴스내용 중 positive_stocks에 있는 각 회사들에 긍정적 영향을 주는 내용. {"회사이름":"긍정적인 이유"}')
    negative_stocks: list[str] = Field(description="뉴스기사에 부정적인 영향을 받는 회사들의 이름들.")
    negative_reason: list[dict[str,str]] = Field(description='뉴스내용 중 negative_stocks에 있는 각 회사들에 부정적 영향을 주는 내용. {"회사이름":"부정적인 이유"}')
    summary: str = Field(description="뉴스 기사 요약")

parser = JsonOutputParser(pydantic_object=SummarySchema)

prompt = ChatPromptTemplate.from_template(
    template=template,
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Chain 구성
model_name = "gpt-5.2"
model = ChatOpenAI(model=model_name)
chain = prompt | model | parser

### 개별 데이터로 Chain 테스트

In [ ]:
# 입력 형식: title\n뉴스내용

from pprint import pprint

news_idx = 15
sample_news = df['title'].loc[news_idx]+"\n"+df['document'].loc[news_idx]
print(sample_news)

추경호 중기 수출지원 총력 무역금융 40조 확대
앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.


In [27]:
# Chain 실행을 통해 Label 생성
response = chain.invoke({"input":sample_news})


In [28]:
from pprint import pprint
pprint(response)

{'is_stock_related': False,
 'negative_reason': [],
 'negative_stocks': [],
 'positive_reason': [],
 'positive_stocks': [],
 'summary': '한국산 제품에 대한 수입규제를 취하는 국가는 26개국이며 규제 건수는 199건으로 집계됐다. 품목별로 철강·금속과 '
            '화학 비중이 높고, 미국의 특별시장상황(PMS) 및 EU의 탄소국경조정제도(CBAM) 등이 주요 규제로 언급됐다. '
            '산업통상자원부는 수입규제국 조사에 적극 대응하고 비관세장벽 사례 발굴 및 민관 네트워크를 강화하겠다고 밝혔다.'}


## Label 만들기

1. 뉴스제목(title)과 뉴스기사(document)를 합쳐서 입력데이터를 만든다.
2. 1의  입력데이터를 LLM에 요청해서 답변을 받은 뒤 DataFrame에 추가한다.

In [29]:
# K개 샘플링

sample_nums = 100  
sample_df = df.sample(sample_nums).reset_index(drop=True)
sample_df.shape

(100, 7)

In [30]:
# 뉴스제목(title)과 뉴스기사(document)를 합쳐서 프롬프트를 생성

articles = sample_df['title']+"\n"+sample_df['document']


In [ ]:
# articles.apply(함수)

0    광주요 ‘2022 분청화기展’ 개최… 이천 수광리 오름가마에서 탄생한 프리미엄 오브...
1    삼성ENG 사우디 아람코 EPC 파트너 선정\n사우디 국영 석유기업 아람코와 EPC...
2    테슬라 20일 실적 발표한다는데…눈여겨 볼 것은 이것\n2분기 실적·현금흐름 20일...
3    이재용 日 최대 경제단체장과 미팅…한일관계 개선 민간 외교\n이재용 삼성전자 부회장...
4    “한국인의 도전정신 美 주류 사회에 전파”\n김성수 HRCap 대표 한국인 최초 N...
dtype: object

In [32]:
# LLM에 label 생성 요청
label_list = articles.apply(lambda x : chain.invoke({'input':x}))

In [43]:
# 결과 확인
idx = 99
print(articles[idx])
label_list[idx]

신용융자 담보비율 속속 인하…증권가 실효성은 글쎄
교보證 첫 반대매매 완화안 발표 타사도 일제 검토 손실 감당 고민 서울경제 금융감독원이 증시 변동성 완화 방안으로 내놓은 증권사 신용 융자 담보 비율 유지 의무 면제 조치가 시행되면서 각 증권사들이 내규 손질에 착수했다. 교보증권을 필두로 각 증권사들은 가까운 시일 내 반대매매와 관련한 완화 조치를 발표할 예정이다. 한편 해당 조치가 반대매매 완화로 발생하는 리스크를 오롯이 감수해야 하는 증권사의 자율에 맡겨졌다는 점을 고려하면 실효성이 크지 않을 것이라는 지적이 나온다. 4일 금융투자업계에 따르면 이날부터 증권사 신용 융자 담보 비율 유지 의무가 면제되면서 각 증권사들은 내부 규제 손질을 위한 검토에 착수했다. 금융투자업 규정에 따르면 증권사는 투자자에게 신용 융자를 시행할 때 담보를 140% 이상 확보하고 내규에서 정한 비율의 담보를 유지해야 한다. 그러나 최근 증시 급락으로 반대매매가 쏟아지자 금융 당국은 이날부터 3개월간 한시적으로 담보금 유지 비율이 140% 이하로 내려가도 증권사에서 반대매매를 통해 주식을 강제 청산하지 않아도 되도록 규제를 완화했다. 당국의 권고가 내려진 만큼 증권사에서도 관련 규제 손질에 나서는 모습이다. 이날 교보증권은 이번 조치 이후 업계 첫 반대매매 완화안을 내놓았다. 다음 거래일 기준 반대매매 비율이 120 130% 이상인 계좌의 경우 발생분에 대해 1차례에 한해 1일 동안 반대매매를 유예한다는 방침이다. 나머지 증권사들은 내부 규제 조정을 검토 중인 것으로 알려졌다. 한 증권사 관계자는 “오늘 공문이 전해진 만큼 아직 조치에 대해 논의 중”이라며 “시장 환경을 봐가면서 투자자의 요청에 따라 반대매매를 하루 정도 유예해주는 등의 방식으로 조치를 시행해나가는 방향이 될 것”이라고 말했다. 한편 해당 조치가 정해진 기준 없이 증권사의 자율에 맡겨졌다는 점을 고려하면 실효성이 크지 않을 것이라는 지적이 나온다. 증권사 입장에서는 반대매매를 시행하지 않을 경우 주가 하락으로 발생하

{'is_stock_related': True,
 'positive_stocks': ['교보증권'],
 'positive_reason': [{'교보증권': '금융당국의 신용융자 담보비율 유지 의무 면제 시행 이후 업계 최초로 반대매매 완화안을 발표(담보비율 120~130% 이상 계좌에 대해 1일 유예)해 투자자 부담 완화 조치에 선제적으로 대응했기 때문'}],
 'negative_stocks': [],
 'negative_reason': [],
 'summary': '금융감독원이 3개월간 신용융자 담보비율 유지 의무를 한시 면제하면서 증권사들이 반대매매 완화 내규 검토에 착수했다.\n교보증권은 업계 첫 완화안을 내고, 다른 증권사들도 유예 등 방식의 조치를 검토 중이다.\n다만 반대매매를 미실시할 경우 손실을 증권사가 떠안아야 해 자율 방식의 규제 완화 실효성이 크지 않다는 지적이 나온다.'}

In [44]:
# Label 타입 변환:  Dictionary 를 str로 변환.
label_list_str = label_list.apply(lambda x : str(x))

In [47]:
print(label_list_str[20])

{'is_stock_related': False, 'positive_stocks': [], 'positive_reason': [], 'negative_stocks': [], 'negative_reason': [], 'summary': '야놀자가 바스켓펀딩과 협업해 ESG 경영 강화를 위한 트래블 펀딩 캠페인(트래블 소셜기버 프로젝트)을 진행한다. 소셜기버 교육 및 기부 프로젝트 기획을 지원하고, 1·2기 참가자를 모집해 운영비와 상금을 제공한다. 지속 가능한 여행을 통한 지역사회 상생과 사회적 가치 창출을 목표로 한다.'}


In [48]:
# Label을 DataFrame에 "label" 컬럼에 추가.
sample_df['label'] = label_list_str

In [49]:
sample_df.head()

,date,category,press,title,document,link,summary,label
0,2022-07-05 10:49:01,economy,서울경제,광주요 ‘2022 분청화기展’ 개최… 이천 수광리 오름가마에서 탄생한 프리미엄 오브제,서울경제 한국 대표 명품 도자 브랜드 광주요가 오는 7월 6일부터 29일까지 ‘분청...,https://n.news.naver.com/mnews/article/011/000...,광주요가 오는 7월 6일부터 29일까지 이천센터점 2층 전시관에서 한국 대표 명품 ...,"{'is_stock_related': False, 'positive_stocks':..."
1,2022-07-06 21:01:01,economy,서울경제,삼성ENG 사우디 아람코 EPC 파트너 선정,사우디 국영 석유기업 아람코와 EPC 협약식 체결 최성안 오른쪽 삼성엔지니어링 대표...,https://n.news.naver.com/mnews/article/011/000...,삼성 6일 삼성엔지니어링은 사우디 국영기업 아람코와 ‘사우디아라비아 EPC 설계·조...,"{'is_stock_related': True, 'positive_stocks': ..."
2,2022-07-05 17:28:01,economy,매일경제,테슬라 20일 실적 발표한다는데…눈여겨 볼 것은 이것,2분기 실적·현금흐름 20일 발표 상하이 봉쇄로 車인도량 감소 전분기보다 6만대 줄...,https://n.news.naver.com/mnews/article/009/000...,5일 블룸버그에 따르면 테슬라 주가는 올해 들어 약 43.17% 하락해 600달러...,"{'is_stock_related': False, 'positive_stocks':..."
3,2022-07-05 14:22:35,economy,한국경제TV,이재용 日 최대 경제단체장과 미팅…한일관계 개선 민간 외교,이재용 삼성전자 부회장이 지난 4일 한일 재계회의 참석차 방한한 도쿠라 마사카즈 일...,https://n.news.naver.com/mnews/article/215/000...,이재용 삼성전자 부회장이 지난 4일 한일 재계회의 참석차 방한한 도쿠라 마사카즈 일...,"{'is_stock_related': True, 'positive_stocks': ..."
4,2022-07-01 08:53:01,economy,서울경제,“한국인의 도전정신 美 주류 사회에 전파”,김성수 HRCap 대표 한국인 최초 NJBIZ ICON 수상 “아시안 아메리칸 커뮤...,https://n.news.naver.com/mnews/article/011/000...,인 28일 현지 시간 한국인으로는 처음으로 ‘2022 NJZ ICON’을 수상한 글...,"{'is_stock_related': False, 'positive_stocks':..."


# 생성된 데이터셋 저장

1. Local에 pickle 이용해 저장
2. Huggingface data-hub에 upload

## pickle로 저장

In [50]:

import os
os.makedirs("dataset", exist_ok=True)

output_file = "dataset/sample_df.pkl"

sample_df.to_pickle(
    output_file
)

## 허깅페이스에 업로드
1. login
2. Dataset객체.push_to_hub(dataset_id: str)

In [51]:
from huggingface_hub import login
from datasets import Dataset, load_dataset
from dotenv import load_dotenv
import os

load_dotenv()

True

In [52]:
# 로그인
login(os.getenv("HUGGINGFACE_API_KEY"))

In [54]:
# DataFrame을 Dataset으로 변환

dataset = Dataset.from_pandas(sample_df)
dataset

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
    num_rows: 100
})

In [55]:
# train/test set으로 분리
dataset_dict = dataset.train_test_split(test_size=0.1)
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
        num_rows: 90
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
        num_rows: 10
    })
})

In [56]:
# 데이터셋을 Huggingface hub 에 업로드.
dataset_id = "naver_economy_news_stock_instruct_dataset-100_samples"
dataset_dict.push_to_hub(dataset_id)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

...

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md:   0%|          | 0.00/609 [00:00<?, ?B/s]

c:\Documents\SKN21_inst\11_llm_파인튜닝\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\datasets--kgmyh--naver_economy_news_stock_instruct_dataset-100_samples. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/kgmyh/naver_economy_news_stock_instruct_dataset-100_samples/commit/9bb16bf7b43abc17a86a68fbb6186f3814fe39a9', commit_message='Upload dataset', commit_description='', oid='9bb16bf7b43abc17a86a68fbb6186f3814fe39a9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/kgmyh/naver_economy_news_stock_instruct_dataset-100_samples', endpoint='https://huggingface.co', repo_type='dataset', repo_id='kgmyh/naver_economy_news_stock_instruct_dataset-100_samples'), pr_revision=None, pr_num=None)

In [ ]:
# Dataset load
user_id = "kgmyh" # 본인 Huggingface 사용자명 입력
load_dataset_dataset_id = f"{user_id}/{dataset_id}"
print(load_dataset_dataset_id)
load_data = load_dataset(load_dataset_dataset_id)

kgmyh/naver_economy_news_stock_instruct_dataset-100_samples


README.md:   0%|          | 0.00/609 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/213k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/37.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/90 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10 [00:00<?, ? examples/s]

In [58]:
load_data

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
        num_rows: 90
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
        num_rows: 10
    })
})

In [59]:
# 1500개 sample 로 만든 데이터 셋 로드

from datasets import load_dataset
d_id = "kgmyh/naver_economy_news_stock_instruct_dataset"
dataset = load_dataset(d_id)

README.md:   0%|          | 0.00/622 [00:00<?, ?B/s]

c:\Documents\SKN21_inst\11_llm_파인튜닝\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\datasets--kgmyh--naver_economy_news_stock_instruct_dataset. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


data/train-00000-of-00001.parquet:   0%|          | 0.00/2.68M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/315k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1350 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/150 [00:00<?, ? examples/s]

In [60]:
dataset

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
        num_rows: 1350
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary', 'label'],
        num_rows: 150
    })
})

In [61]:
# 확인
trainset = dataset['train']
testset = dataset['test']

In [62]:
idx = 0
trainset[idx]

{'date': '2022-07-05 13:41:15',
 'category': 'economy',
 'press': '뉴스1 ',
 'title': '대전도시공사 한국서비스 품질 우수기업 인증 획득',
 'document': '대전도시공사 김홍준 경영부사장 오른쪽 이 한국서비스진흥협회 관계자로부터 인증서를 받은 뒤 기념촬영을 하고 있다. 대전도시공사 제공 © 뉴스1 대전 뉴스1 김경훈 기자 대전도시공사는 산업통상자원부에서 주최하고 한국서비스진흥협회에서 주관하는 한국서비스 품질 우수기업 SQ 인증을 획득했다고 5일 밝혔다. SQ는 산업통상자원부 국가기술표준원의 평가지표를 기준으로 Δ고객접점 서비스 운영관리 Δ서비스 경영 성과 등 총 7개 항목 25개 지표에 대한 서류 심사 현장 평가 암행 평가 등 객관적 심사를 거쳐 인증서를 수여하고 널리 공표하는 제도다. 공사는 각종 사업을 추진하면서 적극적으로 시민의 목소리를 반영하고 입주민이 스스로 결정하는 주택 관리 착한 임대인 사업 29년 연속 흑자 경영과 이익배당 사회공헌 프로그램 등에서 높은 점수를 받았다. 공사 관계자는 “한국서비스품질 우수기업 인증 위상에 걸맞게 보다 품질 높은 서비스를 시민에게 제공하겠다”고 말했다.',
 'link': 'https://n.news.naver.com/mnews/article/421/0006198155?sid=101',
 'summary': '5일 5일 대전도시공사는 산업통상자원부에서 주최하고 한국서비스진흥협회에서 주관하는 총 7개 항목 25개 지표에 대한 서류 심사 현장 평가 암행 평가 등 객관적 심사를 거쳐 인증서를 수여하고 널리 공표하는 제도인 한국서비스 품질 우수기업 SQ 인증을 획득했다고 밝혔다.',
 'label': "{'is_stock_related': False, 'positive_stocks': [], 'positive_reasons': [], 'negative_stocks': [], 'negative_reasons': [], 'summary': '대전도시공사가 

In [63]:
testset[idx]

{'date': '2022-07-06 16:28:16',
 'category': 'economy',
 'press': '연합뉴스 ',
 'title': '삼성전자 반도체·스마트폰·TV에 상반기 성과급 100% 지급',
 'document': '최대치인 기본급 100%로 결정…생활가전은 제일 낮은 62.5% 삼성전자 서초사옥 연합뉴스 자료사진 서울 연합뉴스 김철선 기자 삼성전자가 반도체와 스마트폰 TV 등 주력 사업부 소속 임직원들에게 올해 상반기 성과급으로 최대치인 기본급 상여 기초금 의 100%를 지급하기로 했다. 6일 업계에 따르면 삼성전자는 이날 오후 사내망을 통해 직원들에게 상반기 사업부별 목표달성장려금 TAI 지급률을 통보했다. TAI는 성과급 중 하나로 매년 상·하반기 한 차례씩 지급되며 사업부 실적을 토대로 사업 부문과 사업부의 평가를 합쳐 최대 월 기본급의 100%를 지급한다. 1년에 한 번 연봉의 최대 50%까지 받을 수 있는 초과이익성과급 OPI 과 함께 삼성전자의 대표적인 성과급 제도다. 반도체 사업을 담당하는 디바이스솔루션 DS 부문의 메모리반도체 사업부 파운드리 사업부 시스템LSI 사업부는 모두 최대치인 100%를 받는다. 스마트폰 사업부인 MX사업부와 네트워크사업부 TV 사업을 담당하는 영상디스플레이 VD 사업부도 역시 최대치인 100%를 받게 됐다. 반면 냉장고와 세탁기 등 제품을 담당하는 생활가전사업부에는 전사 사업부 중 가장 낮은 수준인 62.5%의 지급률이 통보됐다. 최근 원자재 가격 상승과 물류비 인상 수요 위축 등의 영향으로 예상보다 수익성이 신통치 않았던 영향인 것으로 알려졌다. 삼성전자는 다음날인 7일 올해 2분기 잠정 경영실적을 발표하고 오는 8일 사업부별 지급률에 따라 상반기 TAI를 지급할 예정이다.',
 'link': 'https://n.news.naver.com/mnews/article/001/0013293397?sid=101',
 'summary': '삼성전자가 반도체와 스마트폰 TV 등 주력 사업부 소속 임직